In [15]:
import pandas as pd 
import numpy as np
import math
from sklearn import preprocessing
from collections import OrderedDict
import warnings; warnings.filterwarnings("ignore")

In [8]:
train_f = pd.read_csv("./data/train/feature.csv")
train_l = pd.read_csv("./data/train/label.csv")
train = pd.merge(train_f, train_l)

In [3]:
var_list = []
for i in range(10):
    for j in range(10):
        for k in range(10):
            var = 'v' + str(i) + str(j) + str(k)
            var_list.append(var)
var_list

['v000',
 'v001',
 'v002',
 'v003',
 'v004',
 'v005',
 'v006',
 'v007',
 'v008',
 'v009',
 'v010',
 'v011',
 'v012',
 'v013',
 'v014',
 'v015',
 'v016',
 'v017',
 'v018',
 'v019',
 'v020',
 'v021',
 'v022',
 'v023',
 'v024',
 'v025',
 'v026',
 'v027',
 'v028',
 'v029',
 'v030',
 'v031',
 'v032',
 'v033',
 'v034',
 'v035',
 'v036',
 'v037',
 'v038',
 'v039',
 'v040',
 'v041',
 'v042',
 'v043',
 'v044',
 'v045',
 'v046',
 'v047',
 'v048',
 'v049',
 'v050',
 'v051',
 'v052',
 'v053',
 'v054',
 'v055',
 'v056',
 'v057',
 'v058',
 'v059',
 'v060',
 'v061',
 'v062',
 'v063',
 'v064',
 'v065',
 'v066',
 'v067',
 'v068',
 'v069',
 'v070',
 'v071',
 'v072',
 'v073',
 'v074',
 'v075',
 'v076',
 'v077',
 'v078',
 'v079',
 'v080',
 'v081',
 'v082',
 'v083',
 'v084',
 'v085',
 'v086',
 'v087',
 'v088',
 'v089',
 'v090',
 'v091',
 'v092',
 'v093',
 'v094',
 'v095',
 'v096',
 'v097',
 'v098',
 'v099',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v110',
 

In [ ]:
# pd.set_option("display.max_columns",None)
# pd.set_option('display.max_rows',None)
train

In [9]:
X_train = pd.read_csv("./data/train/feature.csv")
y_train = pd.read_csv("./data/train/label.csv")
X_test  = pd.read_csv("./data/test/feature.csv")
y_test  = pd.read_csv("./data/test/label.csv")


In [10]:
v_names = X_train.columns[1:]
df_train = pd.merge(X_train, y_train, on= 'APPLICATION_ID')
df_train.head()

,APPLICATION_ID,v000,v001,v002,v003,v004,v005,v006,v007,v008,...,v992,v993,v994,v995,v996,v997,v998,v999,APPLICATION_DATE,DEFAULT_LABEL
0,44c4e0d49d5bf94efa446e03ecf84ba5,NaN,NaN,NaN,5.528423,NaN,NaN,10.0,0.0,0.016305,...,NaN,NaN,NaN,NaN,41.0,0.0,NaN,0.0,2021-09-14,0
1,993ee0eed4ad96f4f5d6dc30d56bdf94,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,NaN,...,NaN,NaN,NaN,NaN,8.0,0.0,NaN,0.0,2021-07-06,0
2,a9815449c0a385ee0d53305b8cca95c6,NaN,NaN,NaN,5.062508,0.00000,NaN,7.0,0.0,1.039721,...,NaN,NaN,NaN,NaN,22.0,0.0,NaN,0.0,2021-06-10,0
3,4f9dd0d609cee4360e110d56cc35afd9,0.0,NaN,NaN,2.575101,0.44461,NaN,NaN,NaN,0.980110,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2021-07-04,0
4,904de5e19a5ea36e8f4b9f4cf5bdd953,0.0,NaN,0.000473,5.528423,0.00000,NaN,1.0,0.0,0.950612,...,NaN,0.079084,0.415888,NaN,76.0,0.0,3.894232,0.0,2021-06-23,0


In [13]:
df_test = pd.merge(X_test, y_test, on= 'APPLICATION_ID')

In [26]:
# 如果是连续性数据，首先排序分为5组，离散化之后，计算WOE
# 如果是离散型数据，直接分类就好（0-10等）, 计算WOE
label = "DEFAULT_LABEL"
n_bins =  5
B_total = df_train[label].sum() # default, y = 1
G_total = len(df_train[label]) - B_total  # not default, y = 0
B_total_test = df_test[label].sum() # default, y = 1
G_total_test = len(df_test[label]) - B_total_test  # not default, y = 0
diverse_threshold = 30
WOE_list = []
woe_is = []
iv_is = []
PSIs = []
for v_name in v_names:
    data_type = 'discrete'
    df_v = df_train[[v_name, label]]
    df_v.dropna(inplace = True)
    df_v_test = df_test[[v_name, label]]
    df_v_test.dropna(inplace=True)
    
    KBD = preprocessing.KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')

    if len(df_v[v_name].unique()) > diverse_threshold:
        # continuous 离散化
        # https://scikit-learn.org/stable/modules/preprocessing.html#discretization
        data_type = 'continous'
        data = KBD.fit_transform(df_v[[v_name]])
        df_v[v_name] = data  # 离散化
        data = KBD.transform(df_v_test[[v_name]])
        df_v_test[v_name] = data  # 离散化
    else:
        continue
    df_data = pd.pivot_table(df_v, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
#     print(df_data[1])
    A_is = np.array(df_data[1])
    
    df_data = pd.pivot_table(df_v_test, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
    E_is = np.array(df_data[1])
#     print(A_is)
#     print(E_is)
    PSI = 0
    for gg in range(len(A_is)):
        if A_is[gg] == E_is[gg]:
            continue
        PSI += (A_is[gg] - E_is[gg]) * math.log(math.e, A_is[gg] / E_is[gg])
#     print(PSI)
    PSIs.append(PSI)
#     break
    # save
#     print(ratio)
#     print(np.array(ratio))
#     break

#     woe_i = np.array(ratio)
#     woe_is.append(woe_i)
#     iv_i = []
#     print((df_data[0] - df_data[1]) * woe_i[i])
#     break
#     for i in range(len(woe_i)):
#         iv_i.append((df_data[0] - df_data[1]) * woe_i[i])
#     iv_i.append(sum(iv_i))
#     print(iv_i)
#     break
#     iv_is.append(iv_i)
#     tmp_dict = OrderedDict()
#     tmp_dict["name"] = v_name
#     tmp_dict["WOE"] = ratio.sum()
#     tmp_dict['Type'] = data_type
#     WOE_list.append(tmp_dict)
    
#     if len(df_v_test[v_name].unique()) > diverse_threshold:
#         # continuous 离散化
#         # https://scikit-learn.org/stable/modules/preprocessing.html#discretization
#         data_type = 'continous'
#         data = KBD.transform(df_v_test[[v_name]])
#         df_v_test[v_name] = data  # 离散化
#     df_data = pd.pivot_table(df_v_test, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
#     df_data[0] = df_data[0] / G_total   # good y=0
#     df_data[1] = df_data[1] / B_total   # bad  y=1
#     ratio = np.log(df_data[0] / df_data[1])
#     ratio.replace([np.inf, -np.inf], 0, inplace=True)
#     E_is = np.array(df_data[1])
#     print(A_is)
#     print(E_is)
#     PSI = 0
#     for gg in range(len(A_is)):
#         PSI += (A_is[gg] - E_is[gg]) * math.log(math.e, A_is[gg] / E_is[gg])
#     print(PSI)
    

# WOE_result = pd.DataFrame(WOE_list)
# WOE_result.head()

In [27]:
psi_limit = 0.25
count = 0
for item in PSIs:
    if item > psi_limit:
        count += 1
print(count / len(PSIs))

0.56625
